# Imputación Missing Value con Regresión Lineal Simple
##### Mayor información: https://github.com/fmansillaib/python_MTC
##### Personal Page: https://francomansilla.com

#### Paso 1: Cargar la Base de Datos 

In [ ]:
import pandas as pd
df = pd.read_excel(".../BD. MTC.xlsx")
df.head()

##### ---- ~ -----
##### Nota 1: La variable ventas es la variable target, para una posterior modelelo supervisado.
##### Nota 2: Desde la variable gast_x# hasta cost_x# son variables que tiene missing value (NaN) lo cual habra que imputar.
##### ---- ~ -----

#### Paso 2: Análisis de Missing Value

In [ ]:
for i in df.columns.tolist():
    porc_i = round(df[i].isnull().sum()/len(df[i]),4)*100
    print(i+"=", porc_i)

##### ---- ~ -----
##### Nota 3: Si encontramos una variable sobre un estandar, por ejemplo, 15% de missing value, se puede evaluar eliminar la variable.
##### ---- ~ -----

##### Paso 3: Imputación de Missing Value

##### Paso 3.1. Criterios a considerar:
###### 1. Considera variables con 2 o más valores unicos.
###### 2. Considera variables con menos del umbral de missing value; este caso 15%.
###### 3. Considera variables dicotomicas, es decir, contenga 2 valores discretos (0 ó 1) u otra combinación.
###### 4. Considera variables categoricas, es decir, variable contenga 3 o más valores discretos. 
###### 5. Considera variables categoricas con caracter continuo, es decir, variable contenga # o más valores discretos. 

##### ---- ~ -----

##### Paso 3.2. Descripción Imputación
###### La variable ing_x2 tiene 6.72% de missing value. Lo usual en un modelo es: $ventas = \beta_0 + \beta_1*ing _x2$
###### Nota 4: Para realizar la imputación, se utilizara solo la $\textbf{información}$ disponible de $\textbf{ing_x2}$ pero solo de la muestra $\textbf{train}$. 
###### Esto se realizara para realizar el siguiente modelo: $ing_x2 = \beta_0 + \beta_1*ventas$

##### ---- ~ -----

##### Paso 3.3. Código de Imputación

In [ ]:
# df a imputar
df_new = df.copy()

# df con valores sin na y filtro
df_var = df.filter(regex = 'ing|cost' , axis = 1)
# o
var_nd = ['ventas', 'muestra']
#Variables a imputar
var_x = [x for x in df_new.columns if x not in var_nd]

In [ ]:
from sklearn.linear_model import LinearRegression

for i in df_var.columns.tolist():  #ó poner for i in var_x:
    
    umbral_unique = 10 # Scalar
    umbral_nan = 0.1 # Scalar
    target = 'venta' # Columna df.
    muestra = 'muestra' # Columna df.
    ventana = 'Train' # Atributo de la Columna Muestra.
    
    #Criterio 1: Umbral de Missing Value
    porc_i = round(df[i].isnull().sum()/len(df[i]),4)
    
    #Criterio 2: Cantidad de Valores Unicos
    unique_val = df[i].loc[df[muestra]==ventana].nunique()
    
    # Imputación por Regresión variables continuas y categorias de caracter continuas.
    if porc_i <= umbral_nan and unique_val >= umbral_unique:
        
        print('La variable '+i ,'fue imputada con REGRESIÓN LINEAL', 'con',+unique_val,'valores únicos y',porc_i, 'valores perdidos.' )        
        
        # Regresión Lineal 
        y = df[target].loc[(df[i].notnull()) & (df[muestra]==ventana)].values.reshape(-1, 1)
        X = df[i].loc[(df[i].notnull()) & (df[muestra]==ventana)].values.reshape(-1, 1)

        lr = LinearRegression()
        lr.fit(y, X)
        b0 = lr.intercept_
        b1 = lr.coef_
        
        # Calculo de Mediana en Ventas cuando X(i) es Null
        p50 =  df[target].loc[(df[i].isnull()) & (df[muestra]==ventana)].median()
        
        # Valor de X(i) que deberia ir en el NaN de la variable X(i)
        value_p50 = b0 + b1 * p50
        
        # Reemplazo en el dataframe copy los NaN de la variable X(i)
        df_new[i].fillna(value_p50[0,0], inplace = True)
    
    # Elimina variables con MISSING VALUE mayor al Umbral y un único valor en los valores únicos.
    elif porc_i > umbral_nan or unique_val <= 1:
        
        print('La variable '+i ,'fue ELIMINADA por tener',porc_i, 'valores perdidos.' )        
        df_new.drop(i, axis = 1, inplace = True)
        
    # Imputación por Valor Faltantes a varaibles dicotomicas y variables con menos de 15 categorias
    elif unique_val > 1  & unique_val < umbral_unique:
        print('La variable '+i ,'fue imputada por VALOR FALTANTE', 'con',+unique_val,'valores únicos y',porc_i, 'valores perdidos.' )        
        
        #Calculo máximo valor
        value_max = df[i].max()
        #Valor máximo +1
        value_mas1 = value_max + 1
        #Reemplazar NaN por el Valor asignado.
        df_new[i].fillna(value_mas1, inplace = True)
#FIN

In [ ]:
df_new.isnull().sum()

In [ ]:
df_new.head()